In [1]:
# Run this cell to configure the Google Colab runtime

import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk(".") if "dsait4090_project_location" in files), "."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

Mounted at /content/drive
Google Colab: /content/drive/Othercomputers/macbook_pro/fact-checking


In [ ]:
%load_ext autoreload
%autoreload

from src.common import get_device, read_json, DATA_PATH
from src.quantemp_processor import QuantempProcessor, QT_VERACITY_LABELS
from src.models.roberta_large_mnli import RobertaLargeMnliClassifier, RobertaLargeMnliTokenizer
from src.classification_training import ClassificationTraining

import torch
import logging
from torch import nn

# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

logging.basicConfig(level=logging.ERROR)

device = get_device()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA: 1, use NVIDIA A100-SXM4-40GB


In [ ]:
train_data = read_json(f"{DATA_PATH}/raw_data/train_claims_quantemp.json")
val_data = read_json(f"{DATA_PATH}/raw_data/val_claims_quantemp.json")

# limit = 20
# train_data = train_data[:limit]
# val_data = val_data[:limit]

In [ ]:
%autoreload

tokenizer = RobertaLargeMnliTokenizer()
data_processor = QuantempProcessor(tokenizer, claim_decomposition=False)

train_dataset = data_processor.transform(train_data)
val_dataset = data_processor.transform(val_data)

  0%|          | 0/9935 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/3084 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
%autoreload

model = RobertaLargeMnliClassifier('roberta-large-mnli', len(QT_VERACITY_LABELS), 1024, 768, dropout=0.1).to(device)

In [ ]:
%autoreload

training = ClassificationTraining(
    model_name="roberta_large_mnli/basic",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    model=model,
    optimizer=torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8),
    loss_function=nn.CrossEntropyLoss(),
    batch_size=16,
    early_stopping_patience=5,
    device=device,
    random_state=42
)

training.train(epochs=30)

Resuming training from epoch 7

EPOCH 7


train:   0%|          | 0/621 [00:00<?, ?it/s]

eval:   0%|          | 0/193 [00:00<?, ?it/s]

    train accuracy: 0.885
     eval accuracy: 0.725

    avg train loss: 0.295
     avg eval loss: 0.829

Saved checkpoint to models/roberta_large_mnli/basic/07_acc_725

EPOCH 8


train:   0%|          | 0/621 [00:00<?, ?it/s]

...

    train accuracy: 0.955
     eval accuracy: 0.701

    avg train loss: 0.132
     avg eval loss: 1.045

Saved checkpoint to models/roberta_large_mnli/basic/09_acc_701
Early stopping counter: 2/5

EPOCH 10


train:   0%|          | 0/621 [00:00<?, ?it/s]

eval:   0%|          | 0/193 [00:00<?, ?it/s]

    train accuracy: 0.947
     eval accuracy: 0.733

    avg train loss: 0.151
     avg eval loss: 1.016

Saved checkpoint to models/roberta_large_mnli/basic/10_acc_733
Early stopping counter: 3/5

EPOCH 11


train:   0%|          | 0/621 [00:00<?, ?it/s]